# Data Generation code from STORM outputs

## 1) Calling Libraries

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## 2) Calling STORM outputs

In [26]:
df_dep  = pd.read_excel(r'Data/Sim0GaugeTot_1y.xlsx',index_col = 0)
df_int  = pd.read_excel(r'Data/Sim0Gaugeint_1y.xlsx',index_col = 0)
df_dur  = pd.read_excel(r'Data/Sim0Gaugedur_1y.xlsx',index_col = 0)
df_strm = pd.read_excel(r'Data/StormSim0Storm_matrix_1y.xlsx',index_col = 0)

## Data Generation with no spatial information

In [27]:
int_67 = df_strm['Storm Intensity'].quantile(0.67)
int_33 = df_strm['Storm Intensity'].quantile(0.33)
dur_67 = df_strm['Storm Duration'].quantile(0.67)
dur_33 = df_strm['Storm Duration'].quantile(0.33)

person_per_grid = [10]   ## Change this for participants
for l in range(len(person_per_grid)):
    c = 0
    storms = df_strm.shape[0]
    crowd = np.zeros((100000, 7))
    for i in range(1, storms):
        storm_int = df_int.iloc[i, 0:554].values
        storm_dur = df_dur.iloc[i, 0:554].values
        storm_tot = df_dep.iloc[i, 0:554].values
        for j in range(554):
            if storm_int[j] > 5 and storm_dur[j] > 0:
                for k in range(person_per_grid[l]):
                    # Intensity Class
                    if storm_int[j] >= int_67:
                        crowd[c, 1] = 3
                    elif int_33 <= storm_int[j] < int_67:
                        crowd[c, 1] = 2
                    else:
                        crowd[c, 1] = 1

                    # Duration Class (note: storm_dur in minutes, we convert to seconds)
                    duration_sec = storm_dur[j] * 60
                    if duration_sec >= dur_67:
                        crowd[c, 2] = 3
                    elif dur_33 <= duration_sec < dur_67:
                        crowd[c, 2] = 2
                    else:
                        crowd[c, 2] = 1

                    # Storm Center flag
                    if storm_int[j] == df_int.iloc[i, 0:554].values.max():
                        crowd[c, 5] = 1
                    else:
                        crowd[c, 5] = 0

                    # Target classification 
                    int_class = crowd[c, 1]
                    dur_class = crowd[c, 2]

                    if (int_class == 1 and dur_class in [1, 2]) or (int_class == 2 and dur_class == 1):
                        crowd[c, 6] = 1  # Normal
                    elif (int_class == 1 and dur_class == 3) or (int_class == 2 and dur_class in [2, 3]) or (int_class == 3 and dur_class == 1):
                        crowd[c, 6] = 2  # Moderate 
                    else:
                        crowd[c, 6] = 3  # Severe

                    # Record other attributes
                    crowd[c, 0] = c
                    crowd[c, 3] = storm_int[j]
                    crowd[c, 4] = duration_sec
                    c += 1

    trunc_at = np.argmax(np.isclose(crowd[1:, 0], 0.)) + 1
    crowd_out = crowd[:trunc_at, :]
    col = ['Person Id', 'Intensity_class', 'Duration_class', 'Storm_Intensity', 'Storm_duration', "Storm Center", 'Target']
    crowd_out_df = pd.DataFrame(crowd_out, columns=col)


## Data Generation with spatial information

In [ ]:
person_per_grid = [5]    ##Change this for participants
for l in range (len(person_per_grid)):
    c = 0
    storms = df_strm.shape[0]
    crowd = np.zeros((3000000,8))
    for i in range (1 , storms):
        storm_int = df_int.iloc[i,0:554].values
        storm_dur = df_dur.iloc[i,0:554].values
        storm_tot = df_dep.iloc[i,0:554].values
        for j in range (554):
            if storm_int[j] > 5 and storm_dur[j] > 0:
                for k in range (person_per_grid[l]):
                    if storm_int[j] >= int_67 :
                        crowd[c,1] = 3

                    elif int_33 <= storm_int[j] < int_67:
                        crowd[c,1] = 2
                    else:
                        crowd[c,1] = 1
                    if storm_dur[j]*60 >= dur_67:
                        crowd[c,2] = 3
                    elif dur_33 <= storm_dur[j]*60 < dur_67:
                        crowd[c,2] = 2
                    else: 
                        crowd[c,2] = 1
                    if storm_int[j] == df_int.iloc[i,0:554].values.max() :
                        crowd[c,5] = 1
                    else:
                        crowd[c,5] = 0
                    if storm_int[j] >= int_67 and storm_dur[j]*60 >= dur_67:
                        crowd[c,6] = 3
                    elif int_33 <= storm_int[j] < int_67 or dur_33 <= storm_dur[j]*60 < dur_67 :
                        crowd[c,6] = 2
                    else:
                        crowd[c,6] = 1
                    crowd[c,0] = j  # Add for spatial location
                    crowd[c,7] = k
                    crowd[c,3] = storm_int[j]
                    crowd[c,4] = storm_dur[j]*60
                    c += 1

    trunc_at = np.argmax(np.isclose(crowd[1:, 1], 0.)) + 1
    crowd_out = crowd[:trunc_at, :]
    col = ['Gauge_Id', 'Intensity_class', 'Duration_class','Storm_Intensity','Storm_duration', "Storm Center", 'Target','Person_id']
    crowd_df = pd.DataFrame(crowd_out,columns = col)
    crowd_df['Gauge_Id'] = crowd_df['Gauge_Id'].astype('int32')
    crowd_df['Person_id'] = crowd_df['Person_id'].astype('int32')
    crowd_df['Gauge_Id'] = crowd_df['Gauge_Id'].astype('str')
    crowd_df['Person_id'] = crowd_df['Person_id'].astype('str')
    crowd_df['Gauge_Id'] = crowd_df['Gauge_Id'].astype('str')
    crowd_df['Person_id'] = crowd_df['Person_id'].astype('str')
    test = crowd_df['Gauge_Id'] + '_' + crowd_df['Person_id']
    crowd_df.insert(loc = 0 , column = 'ID' , value = test)
    crowd_df.to_csv(r'crowd/{}_person_grid_test.csv'.format(person_per_grid[l]))
    